# Draw! - Live Demo smooth transition version (with TensorRT)

The task of the robot is to drive around without collision to draw on a canvas.

The trained network should estimate in which direction it is save to drive:

- forward : both tracks have same speed value
- right : left track has set speed, right track is set 0
- left : right track has set speed, left track is set 0
- turn_right : left track has set speed, right has set -speed
- turn_left : right track has set speed, left has set -speed

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import traitlets
from jetbot import Robot, Camera
from torch2trt import TRTModule
import time

device = torch.device('cuda')

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_trt.pth'))

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

camera = Camera.instance(width=224, height=224, fps=15)

robot = Robot()

class_names = ['forward', 'left', 'right', 'turn_left', 'turn_right']

def low_pass_filter(current_speed, new_speed, alpha):
    return alpha * current_speed + (1 - alpha) * new_speed

def move_robot(direction, speed):
    current_left_motor_speed, current_right_motor_speed = robot.left_motor.value, robot.right_motor.value
    new_left_motor_speed, new_right_motor_speed = 0, 0

    if direction == 'forward':
        new_left_motor_speed = -speed * 0.7
        new_right_motor_speed = -speed * 0.7
    elif direction == 'left':
        new_left_motor_speed = -speed
        new_right_motor_speed = 0
    elif direction == 'right':
        new_left_motor_speed = 0
        new_right_motor_speed = -speed
    elif direction == 'turn_left':
        new_left_motor_speed = speed * 0.9
        new_right_motor_speed = -speed * 0.9
    elif direction == 'turn_right':
        new_left_motor_speed = -speed * 0.9
        new_right_motor_speed = speed * 0.9

    alpha = 0.7
    left_motor_speed = low_pass_filter(current_left_motor_speed, new_left_motor_speed, alpha)
    right_motor_speed = low_pass_filter(current_right_motor_speed, new_right_motor_speed, alpha)

    robot.set_motors(left_motor_speed, right_motor_speed)


last_decision_time = time.time()
decision_timeout = 1.0  # minimum time between direction changes in seconds
last_turning_direction = None

def update(change):
    global last_decision_time, decision_timeout, last_turning_direction
    x = change['new']
    x = preprocess(x)
    y = model_trt(x)
    y = F.softmax(y, dim=1)

    pred_class = torch.argmax(y, dim=1).item()
    pred_prob = torch.max(y).item()

    speed = 1
    threshold = 0.35

    current_time = time.time()
    time_elapsed_since_last_decision = current_time - last_decision_time

    if pred_prob > threshold:
        if class_names[pred_class] in ('turn_left', 'turn_right'):
            if time_elapsed_since_last_decision >= decision_timeout or class_names[pred_class] != last_turning_direction:
                move_robot(class_names[pred_class], speed)
                last_turning_direction = class_names[pred_class]
                last_decision_time = current_time
        else:
            move_robot(class_names[pred_class], speed)
            last_turning_direction = None
    else:
        move_robot('stop', 0)


update({'new': camera.value})  # call the update function once to initialize
camera.observe(update, names='value')  # update function is called asynchronously


## Stop the robot

In [ ]:
camera.unobserve_all()
camera.stop()
time.sleep(1)
robot.stop()